# StarbucksDT_land_price crwaling using Selenium

#### To-Do list
##### 1. open the page
##### 2. read csv file → make dataframe
##### 3. extract all DT stores from dataframe and append it
##### 4. write DT's address on search bar
##### 5. crawling price using selenium
##### 6. extract city name from csv file name
##### 7. save file → set file name using location name

In [1]:
#!pip install selenium
# !pip install bs4

In [1]:
from urllib import request
from selenium import webdriver
import pandas as pd
import numpy as np
import bs4
import requests
import re
import csv
from time import sleep
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys

## 홈페이지에서 스타벅스 DT의 땅값 알아보기

In [2]:
# 다운받은 chromedriver의 path설정과 크롤링주소 설정
path=r"C:\Users\Kavin\Downloads\chromedriver_win32\chromedriver"
driver=webdriver.Chrome(path)
driver.get("http://www.onnara.go.kr/OnnaraServiceMA/onnaraCommon/mainPage.do")
driver.implicitly_wait(5)

In [3]:
# price값에서 ','제외한 숫자데이터만 뽑아옵니다
def get_numeric(data):
    match = re.search('>(.+)<', data)
    if match:
        return int(match.group(1).replace(',',''))    
    return None

In [4]:
def cleantext(content):
    content = re.sub(r'<.*?>', '', str(content))
    content = re.sub(r'· ', '', str(content))
    content = re.sub(r'도로명주소 : ', '', str(content))
    return content

### test 코드입니다

In [47]:
# csv 파일을 읽어오고 데이터프레임에 넣은 후 공시지가 컬럼 새로 추가

city_name_list = ['서울']
price_list = []

for city_name in city_name_list:
    price_list.clear()

    path = "C:/Users/Kavin/Projects/Starbucks_Selenium_Crawler/Starbucks_data/" #저장하고 싶은 경로 앞부분 
    full_path = path + 'starbucks_' + city_name +'.csv' #현재 경로 + 파일명

    # 전체 지정이 담겨져 있는 csv파일 읽어오기
    store = pd.read_csv(full_path, engine = 'python', encoding = 'euc-kr')
    store = store.drop(columns=['Unnamed: 0']) #  Unnamed컬럼 삭제
   
    for address_keyword in store['address']:
        print('*************')
        print(address_keyword)
        
        # 검색창에 주소 입력
        address_search = driver.find_element_by_xpath('//*[@id="totalSerarch"]')
        address_search.clear()
        address_search.send_keys(address_keyword)
        sleep(2)
        address_search.send_keys(Keys.RETURN)
        sleep(2)
        
# 찾기버튼 누르기
#         search = driver.find_element_by_xpath('//*[@id="headerMap"]/section[1]/a[2]')
#         search.click()
#         sleep(5)

        if len(driver.find_elements_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[1]/div/ul/li[1]')) == 0:
            # 만약 검색결과 주소 개수가 없으면 
            print('주소검색불가')
            price_list.append('')
            sleep(2)
            continue

        # 검색결과 주소가 1개라도 있으면 계속
        click_address = driver.find_element_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[1]/div/ul/li/ul') # 결과 주소부분 클릭
        click_address.click()
        sleep(5)

        # 일치하는 주소 찾을동안 반복
        while True:
            source = driver.page_source
            bs = bs4.BeautifulSoup(source,'lxml')

            select_tag1 = '#searchMenu > div.menu_wrap > ul > li.first.actived > div > div > div > div.menu_con.ng-scope > ul > li.actived > div > div.result_list > div > ul > li:nth-of-type('
            select_tag2 = ') > ul > li:nth-of-type(3)'

            list_a = ['1', '2', '3', '4', '5']
            address = []

            for a in list_a:
                select = select_tag1 + a + select_tag2
                el = bs.select_one(select)
                text = cleantext(el)
                address.append(text)
                print([address_keyword, text, text == address_keyword])

                if text == address_keyword:
                    print([text, bs.select('#infoJiga')])
                    price_list.append(bs.select('#infoJiga'))
                    
                    break
                elif text == 'None':
                    break

            if text == address_keyword:
                break
            elif text == 'None':
                print('일치주소없음')
                price_list.append('')
                break
            elif len(address) == 5:
                try: 
                    driver.find_element_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[2]/ul/li[8]/a').click()

                except NoSuchElementException as e:
                    driver.find_element_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[2]/ul/li[6]/a').click()
                    
                    
                

        # 끝
        
        
        # 각종 에러 처리
        # 5개씩 비교하고 그 다음 버튼 누르기
#         if len(address) == 0:
#             try: 
#                 driver.find_element_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[2]/ul/li[8]/a').click()
#             except NoSuchElementException  as e:
#                 driver.find_element_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[2]/ul/li[6]/a').click()
#         else:
#             continue


        try:
            # 만약 지도 이동이 안된다면 확인 버튼 누르도록
            if len(driver.find_elements_by_class_name('sa-confirm-button-container')) > 0:
                driver.find_element_by_xpath('/html/body/div[3]/div[7]/div/button').click()
                print('주소검색불가')
                price_list.append('')
                sleep(2)
        except NoSuchElementException as e:
                price_list.append(bs.select('#infoJiga'))
                sleep(2)


    # 데이터프레임 안에 넣고, <> 태그 제거
    store['price'] = price_list
    store['price'] = store['price'].astype('str')
    store['price'] = store['price'].apply(get_numeric)


    path2 = "C:/Users/Kavin/Projects/Starbucks_Selenium_Crawler/"
    full_path2 = path2 + 'starbucks_price_' + city_name +'.csv'
    store.to_csv(full_path2, header=True, index=True, encoding='euc-kr')
    
i = i + 1

*************
서울특별시 은평구 통일로 867 (갈현동)


ElementNotVisibleException: Message: element not visible
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)


## 올바른 공시지가 가져오는지 test 코드(수정)

In [12]:
# csv 파일을 읽어오고 데이터프레임에 넣은 후 공시지가 컬럼 새로 추가
city_name_list = ['서울']
price_list = []


for city_name in city_name_list:
    price_list.clear()

    path = "C:/Users/Kavin/Projects/Starbucks_Selenium_Crawler/Starbucks_data/" #저장하고 싶은 경로 앞부분 
    full_path = path + 'starbucks_' + city_name +'.csv' #현재 경로 + 파일명

    # 전체 지정이 담겨져 있는 csv파일 읽어오기
    store = pd.read_csv(full_path, engine = 'python', encoding = 'euc-kr')
    store = store.drop(columns=['Unnamed: 0']) #  Unnamed컬럼 삭제
   
    for i, address_keyword in enumerate(store['address']):
        print('*************')
        print(len(price_list), '{}번: {}'.format(i, address_keyword))
        sleep(5)
        
        
        try:
            pass
        except WebDriverException as e:
            webdriver.ActionChains(driver).send_keys(Keys.F5).perform()
#             elem=driver.find_element_by_tag_name("body")
#             elem.send_keys(Keys.CONTROL+"w")
            continue
        
              
        # 검색창에 주소 입력 후 클릭
        address_search = driver.find_element_by_xpath('//*[@id="totalSerarch"]')
        address_search.clear()
        address_search.send_keys(address_keyword)
        sleep(7)
        address_search.send_keys(Keys.RETURN)

        if len(driver.find_elements_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[1]/div/ul/li[1]')) == 0:
            # 만약 검색결과 주소 개수가 없으면 
            print('주소검색불가')
            webdriver.ActionChains(driver).send_keys(Keys.F5).perform()
            price_list.append('')
            sleep(2)
            continue
            
        if len(driver.find_elements_by_class_name('sa-confirm-button-container')) > 0:
            try:
                webdriver.ActionChains(driver).send_keys(Keys.F5).perform()
                driver.find_element_by_xpath('/html/body/div[3]/div[7]/div/button').click()
                price_list.append('')
                print('통신오류 에러발생')
                continue
            except ElementNotVisibleException as e:
                print("에러남")
                pass
            except NoSuchElementException as e:
                print("새로운 에러")
                pass
#         elif len(driver.find_element_by_xpath('/html/body/div[3]')) > 0:
#             try:
#                 pass
#             except ElementNotVisibleException as e:
#                 print("패스")
#                 pass
#             except WebDriverExcetption as e:
#                 driver.find_element_by_xpath('/html/body/div[3]/div[7]/div/button').click()
#                 print('통신오류 에러발생 또남-페이지고침해야함')
#                 driver.navigate().refresh()
#                 pass

                

            
#         # 검색결과 주소가 1개라도 있으면 계속
#         click_address = driver.find_element_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[1]/div/ul/li/ul') # 결과 주소부분 클릭
#         sleep(5)
    

        
        # 일치하는 주소 찾을동안 반복
        while True:
            source = driver.page_source
            bs = bs4.BeautifulSoup(source,'lxml')

            select_tag1 = '#searchMenu > div.menu_wrap > ul > li.first.actived > div > div > div > div.menu_con.ng-scope > ul > li.actived > div > div.result_list > div > ul > li:nth-of-type('
            select_tag2 = ') > ul > li:nth-of-type(3)'

            list_a = ['1', '2', '3', '4', '5']
            address = []
            

            for a in list_a:
                select = select_tag1 + a + select_tag2
                el = bs.select_one(select)
                text = cleantext(el)
                address.append(text)
                print([address_keyword, text, text == address_keyword])
                sleep(2)

                if text == address_keyword:
                    click_address2 = driver.find_element_by_css_selector(select) 
                    click_address2.click()
                    print('여기까지는 왔음')
                    sleep(7)
                    
                    source = driver.page_source
                    bs = bs4.BeautifulSoup(source,'lxml')
                    
                    print(bs.select('#infoJiga')) 
                    price_list.append(bs.select('#infoJiga'))
                    
                    break
                elif text == 'None':
                    break



            if text == address_keyword:
                break
            elif text == 'None':
                print('지적이동 불가')
                price_list.append('')
                break
            elif len(address) == 5:
                try:
                    # '다음' 버튼 클릭 불가능 시에 다음 주소로 가기
                    next_link_end = driver.find_element_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[2]/ul/li[last()-1]')
                    hi = next_link_end.get_attribute('class')
                    if hi == "ng-scope disabled":
                        price_list.append('')
                        break
                    driver.find_element_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[2]/ul/li[last()-1]/a').click()
                    print("다음페이지 넘겨짐")
                    sleep(3)
                    continue
                except NoSuchElementException as e:
                    price_list.append('')
                    print(price_list)
                    break

                    
            # 만약 지도 이동이 안된다면 확인 버튼 누르도록
            elif len(driver.find_elements_by_class_name('sa-confirm-button-container')) > 0:
                try:
#                     webdriver.ActionChains(driver).send_keys(Keys.F5).perform()
                    driver.find_element_by_xpath('/html/body/div[3]/div[7]/div/button').click()
                    print('주소검색불가')
                    price_list.append('')
                    sleep(2)
                except NoSuchElementException as e:
                    price_list.append(bs.select('#infoJiga'))
                    sleep(2)
                except ElementNotVisibleException  as e:
                    pass
    sleep(3)
    
   


    # 데이터프레임 안에 넣고, <> 태그 제거
    store['price'] = price_list
    store['price'] = store['price'].astype('str')
    store['price'] = store['price'].apply(get_numeric)


    path2 = "C:/Users/Kavin/Projects/Starbucks_Selenium_Crawler/"
    full_path2 = path2 + 'starbucks_price_' + city_name +'.csv'
    store.to_csv(full_path2, header=True, index=True, encoding='euc-kr')
    
i = i + 1

*************
0 0번: 서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스)
['서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스)', '서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스 123동, 상가E동)', False]
['서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스)', '서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스 124동, 상가F동)', False]
['서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스)', '서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스 125동, 상가G동)', False]
['서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스)', '서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스 114동)', False]
['서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스)', '서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스 107동, 상가A동)', False]
다음페이지 넘겨짐
['서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스)', '서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스 120동)', False]
['서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스)', '서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스 114동)', False]
['서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스)', '서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스 상가J동)', False]
['서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스)', '서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스 110동, 상가D동)', False]
['서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스)', '서울특별시 성동구 왕십리로 410 (하왕십리동, 센트라스 126동, 상가H동)', False]
*************
1 1번: 서울특별시 중구 소공로 63 (충무로1가)
['서울특별시

KeyboardInterrupt: 